In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Input,Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.layers import AvgPool2D,Dropout,GlobalAveragePooling2D,concatenate
from tensorflow.keras.models import Model
from keras.datasets import cifar10

In [2]:
def inception(x,filter_1,filter_3_reduce,filter_3,filter_5_reduce,filter_5,filter_pool):
    
    conv1 = Conv2D(filter_1,1,padding='same',strides=1, activation="relu")(x)
    
    conv3a = Conv2D(filter_3_reduce,1,padding='same',strides=1, activation="relu")(x)
    conv3b = Conv2D(filter_3,3,padding='same',strides=1, activation="relu")(conv3a)
    
    conv5a = Conv2D(filter_5_reduce,1,padding='same',strides=1, activation="relu")(x)
    conv5b = Conv2D(filter_5,5,padding='same',strides=1, activation="relu")(conv5a)
    
    conv1a = MaxPooling2D(3,strides=1,padding='same')(x)
    conv1b = Conv2D(filter_pool,1,padding='same',strides=1,activation="relu")(conv1a)
    convInception = concatenate([conv1, conv3b,conv5b,conv1b],axis=3)
    
    return convInception

def auxilliaryModel(x,avgPooling):
    
    x1 = AvgPool2D(avgPooling,strides=3)(x)
    x1 = Conv2D(128,1,padding='same',strides=1,activation="relu")(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024,activation="relu")(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(10,activation='softmax')(x1)
    
    return x1

def model():
    
    inputs = Input(shape=(224, 224, 3)) 
    conv1 = Conv2D(64,7,padding='same',strides=2, activation="relu")(inputs)
    maxpool1 = MaxPooling2D(3,padding='same',strides=2)(conv1)
    
    conv2 = Conv2D(1,1,padding='same', activation="relu")(maxpool1)
    conv3 = Conv2D(192,3,padding='same', activation="relu")(conv2)
    maxpool2 = MaxPooling2D(3,padding='same',strides=2)(conv3)
    
    convInception1 =  inception(x=maxpool2,filter_1=64,filter_3_reduce=96,filter_3=128,filter_5_reduce=16,filter_5=32,filter_pool=32)
    convInception2 =  inception(x=convInception1,filter_1=128,filter_3_reduce=128,filter_3=192,filter_5_reduce=32,filter_5=96,filter_pool=64)
    
    maxpool3 = MaxPooling2D(3,strides=2,padding='same')(convInception2)
    
    convInception3 =  inception(x=maxpool3,filter_1=192,filter_3_reduce=96,filter_3=208,filter_5_reduce=16,filter_5=48,filter_pool=64)
   
    dense1 = auxilliaryModel(convInception3,avgPooling=7)
    
    convInception4 =  inception(x=convInception3,filter_1=160,filter_3_reduce=112,filter_3=224,filter_5_reduce=24,filter_5=64,filter_pool=64)
    convInception5 =  inception(x=convInception4,filter_1=128,filter_3_reduce=128,filter_3=256,filter_5_reduce=24,filter_5=64,filter_pool=64)
    convInception6 =  inception(x=convInception5,filter_1=112,filter_3_reduce=144,filter_3=288,filter_5_reduce=32,filter_5=64,filter_pool=64)
    
    dense2 = auxilliaryModel(convInception6,avgPooling=5)
    
    convInception7 =  inception(x=convInception6,filter_1=256,filter_3_reduce=160,filter_3=320,filter_5_reduce=32,filter_5=128,filter_pool=128)

    maxpool4 = MaxPooling2D(3,strides=2,padding='same')(convInception7)
    
    convInception8 =  inception(x=maxpool4,filter_1=256,filter_3_reduce=160,filter_3=320,filter_5_reduce=32,filter_5=128,filter_pool=128)
    convInception9 =  inception(x=convInception8,filter_1=384,filter_3_reduce=192,filter_3=384,filter_5_reduce=48,filter_5=128,filter_pool=128)
    
    avgPool1 = GlobalAveragePooling2D()(convInception9)
    
    dropout1 = Dropout(0.4)(avgPool1)
    
    dense = Dense(10,activation='softmax')(dropout1)
    
    model = Model(inputs, [dense, dense1, dense2])
    
    return model

In [3]:
model = model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 56, 56, 1)    65          max_pooling2d[0][0]              
_______________________________________________________________________________________

In [4]:
import cv2 
import numpy as np 
from keras.utils import np_utils
from skimage import transform

num_classes = 10

In [5]:
(X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

X_train = X_train[:7000].astype('float32')/ 255.0
Y_train = Y_train[:7000]

In [6]:
# Resize training images
X_train = np.array([(cv2.resize(img, (224,224))) for img in X_train[:,:,:,:]])

In [7]:
# Transform targets to keras compatible format
Y_train = np_utils.to_categorical(Y_train, num_classes)

In [8]:
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
import math

In [9]:
epochs = 25
lr = 0.1

def decay(epoch,steps=100):
  lr = 0.1
  drop = 0.96
  epochs_drop = 8
  lrate = lr * math.pow(drop,math.floor((1+epoch)/epochs_drop))
  return lrate

sgd = SGD(lr=lr,momentum=0.9,nesterov=False)

lr_sc = LearningRateScheduler(decay,verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.20, random_state=42)

In [11]:
history = model.fit(X_train, [y_train, y_train, y_train], validation_data=(X_test, [y_test, y_test, y_test]), epochs=epochs, batch_size=256, callbacks=[lr_sc])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
Epoch 1/25
22/22 [==============================] - 29s 1s/step - loss: 3.6850 - dense_4_loss: 2.3034 - dense_1_loss: 2.3027 - dense_3_loss: 2.3027 - dense_4_accuracy: 0.0989 - dense_1_accuracy: 0.1007 - dense_3_accuracy: 0.0975 - val_loss: 3.6854 - val_dense_4_loss: 2.3040 - val_dense_1_loss: 2.3019 - val_dense_3_loss: 2.3027 - val_dense_4_accuracy: 0.0964 - val_dense_1_accuracy: 0.1171 - val_dense_3_accuracy: 0.1021

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
Epoch 2/25
22/22 [==============================] - 22s 998ms/step - loss: 3.6839 - dense_4_loss: 2.3032 - dense_1_loss: 2.2997 - dense_3_loss: 2.3025 - dense_4_accuracy: 0.0995 - dense_1_accuracy: 0.1152 - dense_3_accuracy: 0.1048 - val_loss: 3.6790 - val_dense_4_loss: 2.3009 - val_dense_1_loss: 2.2941 - val_dense_3_loss: 2.2997 - val_dense_4_accuracy: 0.1021 - val_dense_1_accuracy: 0.1407 - val_dense_3_accuracy: 0.1036

Epoch 00003: Learn